In [ ]:
# The below code assumes a s3 Data connection to the said bucket is established
# and is the standar environment variables are injected to this notebook.

import boto3
import os

# Accessing the environment variable set via Data Connection
AWS_S3_BUCKET = os.environ.get('AWS_S3_BUCKET')
FOLDER_PREFIX = os.environ.get('FOLDER_PREFIX')

# Function to download a folder from S3 without checking for existing files
def download_folder_from_s3(bucket_name, folder_prefix, local_path):
    s3 = boto3.client('s3')
    paginator = s3.get_paginator('list_objects_v2')
    for result in paginator.paginate(Bucket=bucket_name, Prefix=folder_prefix):
        if 'Contents' in result:
            for obj in result['Contents']:
                key = obj['Key']
                if key.endswith('/'):  # Skip directories
                    continue
                dest_file_path = os.path.join(local_path, key[len(folder_prefix):])
                try:
                    os.makedirs(os.path.dirname(dest_file_path), exist_ok=True)
                    s3.download_file(bucket_name, key, dest_file_path)
                    print(f"Downloaded: {key} to {dest_file_path}")
                except botocore.exceptions.ClientError as e:
                    print(f"Error downloading {key}: {e}")

# Define the folder prefix and local directory
LOCAL_DIRECTORY = FOLDER_PREFIX

# Download the folder from S3
download_folder_from_s3(AWS_S3_BUCKET, FOLDER_PREFIX, LOCAL_DIRECTORY)